# Temporal Coordination Capacity Analysis

This notebook reproduces the main analyses from:

**"Temporal Coordination Capacity Reveals Integration Without Reportability: A Two-Stage Model of Conscious Access"**

Cabot, Z. (2025). *Neuroscience of Consciousness*.

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Set style
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
sns.set_style('whitegrid')

# Load derived data
gradient = pd.read_csv('../data/processed/Cons01_complete_gradient.csv')
stage_summary = pd.read_csv('../data/processed/Cons01_stage_summary.csv')
subject_level = pd.read_csv('../data/processed/Cons01_subject_level.csv')
meditation = pd.read_csv('../data/processed/Meditation_Summary.csv')
two_stage = pd.read_csv('../data/processed/TwoStageModel_Summary.csv')

print("Data loaded successfully!")

## 1. The Coordination Gradient

Figure 1 shows coordination capacity (Φ_c) across consciousness states, revealing the central paradox: N3 deep sleep shows HIGHER coordination than wakefulness despite no reportable experience.

In [ ]:
# Display gradient data
print("Coordination Gradient Across States")
print("="*60)
print(gradient.to_string(index=False))

In [ ]:
# Figure 1: Coordination Gradient
fig, ax = plt.subplots(figsize=(12, 6))

# Sort by Φ_c
gradient_sorted = gradient.sort_values('Φ_c')

# Colors by reportability
colors = ['#FF6B6B' if not r else '#4ECDC4' 
          for r in gradient_sorted['Reportable']]

bars = ax.barh(range(len(gradient_sorted)), 
               gradient_sorted['Φ_c'],
               xerr=gradient_sorted['SD'],
               color=colors, 
               edgecolor='black',
               capsize=3)

ax.set_yticks(range(len(gradient_sorted)))
ax.set_yticklabels(gradient_sorted['State'])
ax.set_xlabel('Coordination Capacity (Φ_c)')
ax.set_title('Figure 1: Coordination Capacity Across Consciousness States')

# Add Wake baseline
wake_val = gradient[gradient['State'].str.contains('Wake')]['Φ_c'].values[0]
ax.axvline(x=wake_val, color='gray', linestyle='--', alpha=0.7, label='Wake baseline')

# Legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#4ECDC4', label='Reportable'),
    Patch(facecolor='#FF6B6B', label='Non-reportable')
]
ax.legend(handles=legend_elements, loc='lower right')

# Highlight N3 paradox
ax.annotate('THE N3 PARADOX\nHighest coordination,\nNo reportable experience',
            xy=(0.793, 4), xytext=(0.85, 2),
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=10, color='red',
            bbox=dict(boxstyle='round', facecolor='wheat'))

plt.tight_layout()
plt.savefig('../figures/Fig1_Gradient_reproduced.png', dpi=150)
plt.show()

## 2. The N3 Paradox: Primary Finding

N3 deep sleep shows significantly HIGHER coordination than wakefulness, yet produces no reportable experience. This dissociation is the key empirical finding.

In [ ]:
# Subject-level analysis: N3 vs Wake
wake_data = subject_level[subject_level['state'] == 'Wake']['phi_c']
n3_data = subject_level[subject_level['state'] == 'N3']['phi_c']

print("Table 1: N3 vs Wake Comparison")
print("="*60)
print(f"Wake Φ_c: {wake_data.mean():.3f} ± {wake_data.std():.3f}")
print(f"N3 Φ_c:   {n3_data.mean():.3f} ± {n3_data.std():.3f}")
print()

# Paired t-test (same subjects)
t_stat, p_val = stats.ttest_rel(n3_data.values, wake_data.values)
print(f"Paired t-test: t = {t_stat:.2f}, p = {p_val:.4f}")

# Cohen's d
diff = n3_data.values - wake_data.values
d = np.mean(diff) / np.std(diff, ddof=1)
print(f"Cohen's d: {d:.2f} (very large effect)")

# Consistency
n_higher = np.sum(n3_data.values > wake_data.values)
print(f"Subjects with N3 > Wake: {n_higher}/{len(n3_data)} ({100*n_higher/len(n3_data):.0f}%)")

## 3. The Two-Stage Model

The N3 paradox resolves with a two-stage model:

**Stage 1 (Conscious Moment):** Φ_c(t) > θ_event

**Stage 2 (Reportable):** Stage 1 AND access function available

N3 has HIGH Stage 1 (coordination) but BLOCKED Stage 2 (access).

In [ ]:
# Two-Stage Model Summary
print("Two-Stage Model of Consciousness")
print("="*70)
print(two_stage[['state', 'phi_c', 'reportable', 'access_function', 'interpretation']].to_string(index=False))

In [ ]:
# Visualize Two-Stage Model
fig, ax = plt.subplots(figsize=(10, 6))

# Create 2D plot: Φ_c vs Access
access_map = {'Blocked': 0, 'Impaired': 0.5, 'Variable': 0.75, 'Available': 1, 'Enhanced': 1.2}

for _, row in two_stage.iterrows():
    access_val = access_map.get(row['access_function'], 0.5)
    color = '#4ECDC4' if row['reportable'] == True else '#FF6B6B'
    ax.scatter(row['phi_c'], access_val, s=200, c=color, edgecolor='black', zorder=5)
    ax.annotate(row['state'].replace('_', '\n'), 
                (row['phi_c'], access_val),
                textcoords='offset points',
                xytext=(0, 15), ha='center', fontsize=8)

# Quadrant boundaries
ax.axvline(x=0.65, color='gray', linestyle='--', alpha=0.5)
ax.axhline(y=0.75, color='gray', linestyle='--', alpha=0.5)

# Labels
ax.set_xlabel('Coordination Capacity (Φ_c)')
ax.set_ylabel('Access Function')
ax.set_yticks([0, 0.5, 0.75, 1, 1.2])
ax.set_yticklabels(['Blocked', 'Impaired', 'Variable', 'Available', 'Enhanced'])
ax.set_title('Two-Stage Model: Coordination × Access')

# Quadrant labels
ax.text(0.55, 0.3, 'LOW Φ_c\nNo conscious moment', ha='center', fontsize=9, style='italic')
ax.text(0.55, 1.1, 'LOW Φ_c\nAccess available', ha='center', fontsize=9, style='italic')
ax.text(0.85, 0.3, 'HIGH Φ_c\nNo access\n(N3 PARADOX)', ha='center', fontsize=9, 
        style='italic', color='red', fontweight='bold')
ax.text(0.85, 1.1, 'HIGH Φ_c\nAccess available\n(REPORTABLE)', ha='center', fontsize=9, 
        style='italic', color='green', fontweight='bold')

plt.tight_layout()
plt.savefig('../figures/TwoStageModel.png', dpi=150)
plt.show()

## 4. Meditation Expertise Gradient

Coordination capacity is TRAINABLE: experts show higher Φ_c than novices, and cessation achieves near-maximal coordination with perfect temporal balance (τ ≈ 1).

In [ ]:
# Meditation summary
print("Meditation Expertise Gradient")
print("="*60)
print(meditation.to_string(index=False))

In [ ]:
# Figure 3: Meditation gradient
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Panel A: Φ_c by expertise
fig3_data = pd.read_csv('../data/figures/Figure3_meditation_data.csv')

colors = ['#808080', '#87CEEB', '#4169E1', '#FFD700']
bars = ax1.bar(fig3_data['group'], fig3_data['phi_c'], 
               yerr=fig3_data['phi_c_sd'],
               color=colors, edgecolor='black', capsize=5)

ax1.set_ylabel('Coordination Capacity (Φ_c)')
ax1.set_title('A. Trainable Coordination')
ax1.set_ylim(0.6, 1.0)

# Add trend arrow
ax1.annotate('', xy=(3.5, 0.95), xytext=(0.5, 0.68),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))
ax1.text(2, 0.78, 'Training\neffect', ha='center', fontsize=10, color='green')

# Panel B: Temporal orientation (τ)
ax2.bar(fig3_data['group'], fig3_data['tau'],
        yerr=fig3_data['tau_sd'],
        color=colors, edgecolor='black', capsize=5)

ax2.axhline(y=1.0, color='red', linestyle='--', label='Perfect balance (τ=1)')
ax2.set_ylabel('Temporal Orientation (τ = Φ_f/Φ_d)')
ax2.set_title('B. Temporal Balance')
ax2.legend(loc='upper left')

# Annotate cessation
ax2.annotate('Cessation achieves\nperfect balance', xy=(3, 1.01), xytext=(2.2, 1.1),
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=10, color='red')

plt.tight_layout()
plt.savefig('../figures/Fig3_Meditation_reproduced.png', dpi=150)
plt.show()

## 5. Summary: The Two-Stage Model

The core theoretical contribution is the **Two-Stage Model of Conscious Access**:

### Stage 1: The Conscious Moment
```
Conscious moment at time t  ⟺  Φ_c(t) > θ_event
```
Coordination capacity constitutes the conscious moment itself—the transient synthesis of past and future.

### Stage 2: Reportability
```
Reportable  ⟺  (Φ_c(t) > θ_event) ∧ (A(t) > θ_access)
```
A separate access function determines which moments become reportable, memorable, and narratively integrated.

### Key Insight
Consciousness is not LOST in N3 deep sleep—its products cannot enter the workspace that supports reportable experience. The coordination is there; the access is blocked.

In [ ]:
print("\n" + "="*70)
print("THE SENTENCE TO REMEMBER")
print("="*70)
print("""
"Consciousness corresponds to the coordination phase itself: the transient 
synthesis of past and future influences quantified by Φ_c(t); what varies 
across states is not the presence of these conscious moments, but whether 
their products become globally accessible."
""")

---

## References

1. Cabot, Z. (2025). Temporal Coordination Capacity Reveals Integration Without Reportability. *Neuroscience of Consciousness*.
2. Tononi, G. et al. (2016). Integrated Information Theory. *Nature Reviews Neuroscience*.
3. Dehaene, S. & Changeux, J.P. (2011). Global Workspace Theory. *Neuron*.
4. McGilchrist, I. (2009). *The Master and His Emissary*.